In [2]:
import folium
import pandas as pd
import pickle
import os
import h3
from label_samples_time_hexa import label_samples
from vizualize import *
from func import *

In [3]:
matrix = pd.read_pickle("/home/jaro/BINP29/Project_Eran/1_dist_matrix/eucl_dist.pkl")

In [5]:
time_bins = 11
resolution = 3
same_age_range = True
df = label_samples("/home/jaro/BINP29/Project_Eran/", time_bins, resolution, same_age_range)
time_bins = rename_time_bins(df)

In [49]:
time_bins_dist = calc_dist_time_bin(df, matrix)

In [56]:
selected_time_bin = time_bins[7]
time_bin = time_bins_dist[selected_time_bin]
time_bin, gen_distances_pred = scale_distances(time_bin, resolution=resolution)
# get the hexagons with there internal distance and the distance values for the selected time bin
time_bin, hexagons = get_hexagons(time_bin)

In [59]:
lines = get_distance_lines(time_bin)
m = draw_sample_hexagons(hexagons)
m = draw_barriers(lines, m)
m

In [57]:
threshold = 0.4
isolated_hex, barrier_lines, barrier_hex = get_isolated_hex_and_barriers(time_bin, hexagons, threshold, allowed_distance=10)

print(f"Number of isolated hexagons: {len(isolated_hex)}")
print(f"Number of barrier lines: {len(barrier_lines)}")
print(f"Number of barrier hexagons: {len(barrier_hex)}")

Number of isolated hexagons: 26
Number of barrier lines: 10
Number of barrier hexagons: 177


In [61]:
# get the index of the time bin of interest
time_bin_index = time_bins.index(time_bins[7])
# get the closest population for each isolated hexagon
closest_populations, new_isolated_hex = find_closest_population(df, time_bin_index, isolated_hex, matrix, threshold, gen_distances_pred, 3)
print(f"Number of isolated hexagons with no migration: {len(new_isolated_hex)}")

Number of isolated hexagons with no migration: 23


In [62]:
imputed_hex = impute_missing_hexagons(barrier_hex)

In [63]:
#m = draw_hexagons(new_isolated_hex, color = "red", m = m, opacity=0.7, value="No migration")
m = draw_hexagons_with_values(barrier_hex, m)
m = draw_hexagons_with_values(imputed_hex, m, imputed=True)
m = draw_sample_hexagons(hexagons, m)
m = draw_barriers(barrier_lines, m)
m = draw_migration_for_time_bin(closest_populations, m)
m